<a href="https://colab.research.google.com/github/francescostreet/-AN2DL-2025-First-challenge/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
# from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import copy
import shutil
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

PyTorch version: 2.8.0+cu126
Device: cpu


In [2]:
# Load dataset

X_train = pd.read_csv('pirate_pain_train.csv')
y_train = pd.read_csv('pirate_pain_train_labels.csv')

X_test = pd.read_csv('pirate_pain_test.csv')

In [4]:
# Define column names for the dataset
column_names = ['sample_index', 'time', 'pain_survey_1', 'pain_survey_2', 'pain_survey_3', 'pain_survey_4', 'n_legs', 'n_hands', 'n_eyes', 'joint_00']

# Read the dataset into a DataFrame with specified column names
df = pd.read_csv('pirate_pain_train.csv', header=None, names=column_names)

# Remove rows with any missing values
df.dropna(axis=0, how='any', inplace=True)

# Print the shape of the DataFrame
print(f"DataFrame shape: {df.shape}")

# Display the first 10 rows of the DataFrame
df.head(10)

DataFrame shape: (105761, 10)


sample_index  \
sample_index time pain_survey_1 pain_survey_2 pain_survey_3 pain_survey_4 n_legs n_hands n_eyes joint_00           joint_01           joint_02           joint_03           joint_04           joint_05           joint_06           joint_07           joint_08           joint_09           joint_10            joint_11            joint_12           joint_13               joint_14               joint_15               joint_16               joint_17               joint_18               joint_19               joint_20                              joint_21   
000          0    2             0             2             1             two    two     two    1.0947052308906111 0.9852806323892689 1.0183017685309896 1.0103846219973498 0.9717169667677581 1.0222626970207267 0.901754688096325  0.9996588076563115 0.7129890259094573 1.0501418311133923 0.5295547622945486  0.4473700071620395  1.091045860897634  0.0                    3.0539757619950095e-07 4.7914302751208566e-06 4.402450698163696e-06  0.0                    1.6849528758887604e-05 1.2103686199933038e-05 6.140248225873454e-07    3.499557792481375e-06   
             1    2             2             2             2             two    two     two    1.135183103536129  1.0211747000619738 0.9943433725932543 1.052363882715683  0.9999441405465478 1.012395036980465  0.923340576193706  1.0358501537217646 0.7226847196091615 1.0603133408550205 0.44680991176983303 0.41443183501500275 1.0458617871650786 1.2325046641239027e-05 4.806347286868409e-06  0.0                    3.8751338174573535e-06 8.706925123652644e-06  0.0                    1.0194172618414565e-05 1.9319783908446927e-06   3.976952213431784e-07   
             2    2             0             2             2             two    two     two    1.0807448109145028 0.9628423568573223 1.0095875703922053 0.9771686446308178 0.9847399141416346 1.019929688699317  0.9765671074880842 1.0727511484853933 0.668043044142936  1.0114104124023555 0.43249864699822294 0.43153480854491194 1.0882210916846609 0.0                    3.125992543870344e-07  0.0                    1.035689911330332e-06  0.0                    9.727752212477372e-06  2.0240181089146394e-06 5.730837816958964e-07   1.5338202764856223e-07   
             3    2             2             2             2             two    two     two    0.9380165318378043 1.081592270878279  0.9980205532045711 0.9872828163155588 0.9241606400066662 1.0026415359849703 0.8309818845474468 1.0807549911684773 0.7020846376287088 1.047223091467062  0.4788060971456342  0.4206648391990636  1.0968324679878347 0.0                    3.1618889619588626e-07 0.0                    2.5045319889005086e-06 3.4564764188112654e-05 0.0                    6.883125683618709e-06  3.677485882417406e-05   1.0068651685544181e-05   
             4    2             2             2             2             two    two     two    1.0901849482346806 1.0321446370205805 1.0087097025989937 0.963657611812324  1.01629057713775   1.0313006090643326 0.9560081721226316 0.9880225833649314 0.7121966006151591 1.0447308388816443 0.45290630069456983 0.4765371874884908  1.1039677783007522 0.0                    3.6112449603849616e-07 1.8837532691671258e-06 1.0377896638281432e-06 0.0                    2.9797576830971937e-05 2.034571423859658e-06  2.0379387724607624e-05   4.437265668725749e-06   
             5    2             0             2             1             two    two     two    1.1460314612288625 1.0220919949337242 0.975502922491714  0.9703033927783636 1.0924268139421016 1.0267486812509379 0.9030129466841167 1.0195066405570703 0.6462276037756015 1.056162106431079  0.5470641611525502  0.46470935866864155 1.1433193525698642 0.0                    3.2334535499303917e-07 1.2188906419334232e-06 1.0385433432397744e-06 0.0                    3.263957907020099e-05  1.6006029591871748e-05 2.0607825512760706e-05  1.0731674090588247e-06   
             6    2             1             2             1             two    two     two